# JETANK Servos

JETANK is an open source robot product based on jetbot. It adds 5 servos and a pair of crawlers on the basis of jetbot, which gives it a camera that can pitch and a robotic arm that can grip objects.

Before controlling the robotic arm, make sure that the servos are connected with the driver board and the initial position of each servo has been correctly calibrated according to the content of the tutorial. When the servos are connected correctly and powered on, there will be an obvious sense of damping when turning the servos.

In the next document, we will use Python to control the servos.


### Importing the TTLServo class

First, we need to import the library used to control the servos. Before running this code, please note that ttyTHS1 (the serial port for communication between Jetson Nano and the servos) has been occupied by the Jetson Nano system itself by default. If the installation script executes everything normally, it will shut down the service that previously occupied ttyTHS1, and the script will automatically add the 0666 permissions to ttyTHS1.



If the previous installation script fails to execute, you may encounter a Permission Denied error when using ttyTHS1 here. If you encounter this error, the following is the solution:
- Click the plus sign in the upper left corner of JupyterLab, a new Launcher tab will open, and Terminal in this tab will open the Jetson Nano console.
- Enter the following in the console and press Enter.
    > sudo chmod 666 /dev/ttyTHS1
- Enter Jetpack's default password `jetbot`, and press Enter to confirm the modification permissions.
- Right-click on `JETANK_1_servos.ipynb` on the left, and click `Shut Down Kernal`.
- Close the current `JETANK_1_servos.ipynb` tab and double-click on the left side `JETANK_1_servos.ipynb` again to run a new Kernal.

Then you can select the following code and press Ctrl+Enter to import the library used to control the servos. The following prompt will appear:
Succeeded to open the port
Succeeded to change the baudrate


In [1]:
from SCSCtrl import TTLServo

Succeeded to open the port
Succeeded to change the baudrate


Next, we use the simplest servo control function `servoAngleCtrl()` to control the servos. This function can be used to control the rotation of a certain servo. Four parameters are required when calling this function:
`servoAngleCtrl(servoID, angleInput, direction, speed)`

> **servoID:** The number of the controlled servo. The servo number for the horizontal direction (Pan) rotation of the robotic arm (camera) is No. 1; the servo number for the pitching movement of the root of the robotic arm is No. 2; the servo number for pitching movement of the middle joint of the robotic arm The steering gear is the No. 3; the No. 4 servo is used to grip the objects; the No. 5 servo is for the camera's Tilt movement.

> **angleInput:** Control the rotation angle of the servo. Theoretically, the range can be +-150. In fact, considering the interference of the mechanical structure, the rotation range will be limited. It should be noted here that the angle of angleInput is relative to the center(initial) position of the steering gear. If you want the steering gear to rotate to the center(initial) position, let the value of angleInput be 0.

> **direction:** The value of this parameter can be 1 or -1. This parameter is used to adjust the rotation direction of the servo, such as `TTLServo.servoAngleCtrl(4, -40, 1, 150)` and `TTLServo.servoAngleCtrl(4, 40, -1) , 150)` are both control the servo to rotate to the same position.

> **speed:** This parameter is used to control the rotation speed of the servo. The larger the value, the faster the rotation speed. The range is 1-1500, but it should be noted that when this parameter is 0, it also represents the maximum rotation speed.

Run the code cell below to make all the servos rotate to the initial position at a speed of 150.

In [2]:
TTLServo.servoAngleCtrl(1, 0, 1, 150)
TTLServo.servoAngleCtrl(2, 0, 1, 150)
TTLServo.servoAngleCtrl(3, 0, 1, 150)
TTLServo.servoAngleCtrl(4, 0, 1, 150)
TTLServo.servoAngleCtrl(5, 0, 1, 150)

512

### Resolved Motion Control

Since the position of the chuck of the robotic arm is controlled by the coupling of two servos, if we control the rotation of the two servos separately, it will be difficult to control the position of the chuck of the robotic arm. Therefore, an inverse kinematics function is used to calculate the position of the chuck. For the rotation angle of the two coupled servos, you only need to input the relative coordinate value `(x, y)` of the position where the chuck you want to control the motion of the robotic arm.

Run the following code to control the movement of the mechanical arm chuck to the position of `(100,0)`, 100 is the X value, the larger the X value, the forward the chuck position; 0 is the Y value, the larger the Y value, the higher the position. The unit is millimeters.

In [3]:
TTLServo.xyInput(100, 0)

[-114.6243183521641, 55.377838160268176]

In order to show the relationship between X and Y values more intuitively, we let the robotic arm move from `(100, 0)` to `(200, 0)` step by step, and then back to the position `(150, 0)`.
The movement here changes the X parameter of the position.
Run the following code while observing the movement of the robotic arm.

In [4]:
import time

# Move to position（100，0）
TTLServo.xyInput(100, 0)
time.sleep(1)

# from（100，0）to（200，0）step by step
for i in range(100, 201):
    TTLServo.xyInput(i, 0)
    time.sleep(0.01)
time.sleep(1)

# Smooth back and forth movement between（100，0）and（100，100）,The third parameter is the time required to move to the position, in seconds.
for i in range(0,4):
    TTLServo.xyInputSmooth(100, 0, 1)
    time.sleep(2)
    TTLServo.xyInputSmooth(100, 100, 1)
    time.sleep(2)

# Move to position（150，0）
TTLServo.xyInput(150, 0)
time.sleep(1)

The following code only changes the Y value, run the following code while observing the movement of the robotic arm.

In [5]:
# Move to position（150，-50）
TTLServo.xyInput(150, -50)
time.sleep(1)

# from（150，-50）to（150，50）step by step
for i in range(-50, 51):
    TTLServo.xyInput(150, i)
    time.sleep(0.01)
time.sleep(1)

# Move to position（150， 0）
TTLServo.xyInput(150, 0)
time.sleep(1)


When we are using the servos, sometimes we just want it to start rotating and the rotation angle is not determined. For example, we can use `servoAngleCtrl(1, 80, 1, 150)` to control the camera to start rotating to the right, and then call `servoStop(1)` to stop servo No. 1. The `80` above also can be used to set the limit angle.

In [6]:
TTLServo.servoAngleCtrl(1, 80, 1, 150)
time.sleep(2)
TTLServo.servoStop(1)

The above is the primary use of the steering gear.